Install Required Libraries

In [1]:
# Install necessary libraries
!pip install transformers datasets scikit-learn gradio --quiet

Load AG News CSVs

In [2]:
!wget https://raw.githubusercontent.com/mhjabreel/CharCnn_Keras/master/data/ag_news_csv/train.csv
!wget https://raw.githubusercontent.com/mhjabreel/CharCnn_Keras/master/data/ag_news_csv/test.csv


--2025-07-14 08:41:35--  https://raw.githubusercontent.com/mhjabreel/CharCnn_Keras/master/data/ag_news_csv/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29470338 (28M) [text/plain]
Saving to: ‘train.csv.4’

train.csv.4         100%[===================>]  28.10M   105MB/s    in 0.3s    

2025-07-14 08:41:36 (105 MB/s) - ‘train.csv.4’ saved [29470338/29470338]

--2025-07-14 08:41:36--  https://raw.githubusercontent.com/mhjabreel/CharCnn_Keras/master/data/ag_news_csv/test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length

Load AG News CSVs

In [3]:
import pandas as pd

# Load CSV files
train_df = pd.read_csv("train.csv", header=None)
test_df = pd.read_csv("test.csv", header=None)

# Rename columns
train_df.columns = ["label", "title", "text"]
test_df.columns = ["label", "title", "text"]

# Labels in this CSV version start from 1, so subtract 1 to match BERT's expected format (0-3)
train_df["label"] = train_df["label"] - 1
test_df["label"] = test_df["label"] - 1

train_df.head()


,label,title,text
0,2,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,2,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
2,2,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
3,2,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
4,2,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."


Convert to Dataset & Tokenize

In [4]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)


Convert to Dataset & Tokenize

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=128)

train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

Load Pretrained Model

In [6]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=4)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Define Custom Metrics Function

In [17]:
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "eval_accuracy": accuracy_score(labels, preds),
        "eval_f1": f1_score(labels, preds, average="weighted")
    }


Define TrainingArguments and Trainer

In [19]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    save_total_limit=1,
    load_best_model_at_end=True,
    report_to="wandb",  # or use "none" to disable W&B
    run_name="bert-agnews-run",
    metric_for_best_model="eval_accuracy",
    greater_is_better=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics  # ✅ Ensures eval_accuracy is returned
)


In [9]:
!pip install numpy==1.26.4 --quiet


Train the Model

In [20]:
trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.162000,0.274756,0.924474,0.924774
2,0.130200,0.274107,0.933684,0.933571
3,0.071800,0.335902,0.935263,0.935264


TrainOutput(global_step=22500, training_loss=0.12131530219184028, metrics={'train_runtime': 8486.7784, 'train_samples_per_second': 42.419, 'train_steps_per_second': 2.651, 'total_flos': 2.368042020864e+16, 'train_loss': 0.12131530219184028, 'epoch': 3.0})

Saving the model

In [21]:
model.save_pretrained("./bert_agnews_model")
tokenizer.save_pretrained("./bert_agnews_model")


('./bert_agnews_model/tokenizer_config.json',
 './bert_agnews_model/special_tokens_map.json',
 './bert_agnews_model/vocab.txt',
 './bert_agnews_model/added_tokens.json',
 './bert_agnews_model/tokenizer.json')

Making a zip folder

In [24]:
import shutil
shutil.make_archive("bert_agnews_model", "zip", "./bert_agnews_model")


'/content/bert_agnews_model.zip'

Downloading the model zip folder

In [25]:
from google.colab import files
files.download("bert_agnews_model.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Evaluate Model

In [22]:
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted")
    }

trainer.compute_metrics = compute_metrics
metrics = trainer.evaluate()
print(metrics)


{'eval_loss': 0.3359023630619049, 'eval_accuracy': 0.9352631578947368, 'eval_f1': 0.9352641717585167, 'eval_runtime': 58.0833, 'eval_samples_per_second': 130.847, 'eval_steps_per_second': 2.049, 'epoch': 3.0}


Gradio Demo

In [27]:
import gradio as gr
import torch

labels = ['World', 'Sports', 'Business', 'Sci/Tech']

def classify_news(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)

    # Match model and input device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    probs = torch.nn.functional.softmax(outputs.logits, dim=1).cpu().numpy()[0]
    return {labels[i]: float(probs[i]) for i in range(4)}


gr.Interface(
    fn=classify_news,
    inputs="text",
    outputs="label",
    title="📰 News Topic Classifier (BERT)"
).launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9ee866dc8942eb8c7c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
